# Exploratory Code

For each model I suggest we use:

|Model Name|Number of Features|Names of Features
|--------------|---------|---------
|Comb Model|25 |'mu_plus_ProbNNk', 'mu_plus_ProbNNe', 'mu_plus_ProbNNp', 'mu_minus_ProbNNk', 'mu_minus_ProbNNe', 'mu_minus_ProbNNp', 'K_ProbNNk', 'K_ProbNNpi', 'K_ProbNNmu', 'K_ProbNNe', 'Pi_ProbNNk', 'Pi_ProbNNmu', 'Pi_ProbNNe', 'Pi_ProbNNp', 'Pi_PT', 'Pi_IPCHI2_OWNPV', 'B0_ENDVERTEX_CHI2', 'B0_PT', 'Kstar_M', 'J_psi_M', 'B0_IPCHI2_OWNPV', 'B0_DIRA_OWNPV', 'B0_OWNPV_X', 'B0_OWNPV_Y', 'q2'
|Peaking Model (Background Trained Together) |13 | 'K_ProbNNk', 'K_ProbNNpi', 'K_ProbNNp', 'Pi_ProbNNk', 'Pi_ProbNNpi', 'Pi_ProbNNp', 'Pi_PT', 'Pi_PX', 'Pi_IPCHI2_OWNPV', 'B0_M', 'Kstar_M', 'costhetal', 'costhetak'

For Peaking Model (Background Trained Separately):

|Model Name|Number of Features|Names of Features
|--------------|---------|---------
|jpsi_mu_k_swap|4 |'mu_plus_ProbNNk', 'mu_plus_ProbNNmu', 'K_ProbNNk', 'K_ProbNNmu'
|jpsi_mu_pi_swap  |5|'mu_minus_ProbNNpi', 'mu_minus_ProbNNmu', 'Pi_ProbNNpi', 'Pi_ProbNNmu', 'costhetal'
|k_pi_swap |5|'K_ProbNNk', 'K_ProbNNpi', 'Pi_ProbNNk', 'Pi_ProbNNpi', 'Pi_ProbNNp'
|Kmumu|7|'Pi_PT', 'Pi_PX', 'Pi_IPCHI2_OWNPV', 'B0_M', 'B0_ENDVERTEX_CHI2', 'B0_OWNPV_Y', 'costhetak'
|Kstarp_pi0 |7|'Pi_PT', 'Pi_PX', 'Pi_IPCHI2_OWNPV', 'B0_M', 'B0_ENDVERTEX_CHI2', 'Kstar_M', 'Kstar_ENDVERTEX_CHI2'
|phimumu |7|'Pi_ProbNNk', 'Pi_ProbNNpi', 'Pi_ProbNNp', 'Pi_PT', 'B0_M', 'Kstar_M', 'costhetak'
|pkmumu_piTok_kTop |6|'K_ProbNNk', 'K_ProbNNp', 'Pi_ProbNNk', 'Pi_ProbNNpi', 'B0_M', 'B0_OWNPV_Y'
|pKmumu_piTop |8|'Pi_ProbNNk', 'Pi_ProbNNpi', 'Pi_ProbNNp', 'Pi_P', 'Pi_PE', 'B0_M', 'Kstar_M', 'B0_OWNPV_Y'



We see grouping in the ROC_AUC_SCORE, to avoid over fitting when the score stops changing drastically we potentially can stop including more features.

The ROC_AUC_SCORE is generally higher for separately trained models as they contain less data.






In [14]:
import pandas as pd

total_data = pd.read_csv('../data/total_dataset.csv')

In [15]:
def remove_columns(dataset, reject_rows = []):
    #'B0_M', 'J_psi_M', 'q2','Kstar_M'
    dataset_modified = dataset.copy()
    dataset_modified.drop(columns=['Unnamed: 0.1','Unnamed: 0', 'year', 'B0_ID', 'B0_ENDVERTEX_NDOF','J_psi_ENDVERTEX_NDOF', 'Kstar_ENDVERTEX_NDOF'], inplace=True)
    if 'Unnamed: 0.2' in dataset_modified.columns:
        dataset_modified.drop(columns=['Unnamed: 0.2'], inplace=True)

    columns_list = dataset_modified.columns.tolist()
    for x in reject_rows:
        if x in columns_list:
            dataset_modified.drop(columns=[x],inplace = True)
    
    return dataset_modified

In [16]:
def high_corrolation_list_peaking_separate(num,name_of_background):
    corrolation = pd.read_csv('Peaking_Separate_Correlation/continuous_f1_score_{}.csv'.format(name_of_background))
    cols = corrolation.columns.tolist()
    values = {}
    for x in cols:
        values[x] = float(corrolation[x])

    values = dict(sorted(values.items(), key=lambda item: item[1], reverse=True))

    array_list = []
    for idx, key in enumerate(values):
        if(idx > num-1):
            array_list.append(key)
    return array_list

In [26]:
def high_corrolation_list_peaking_together(num):
    corrolation = pd.read_csv('Peaking_Together_Correlation/continuous_f1_score_peaking_together.csv')
    cols = corrolation.columns.tolist()
    values = {}
    for x in cols:
        values[x] = float(corrolation[x])

    values = dict(sorted(values.items(), key=lambda item: item[1], reverse=True))
    array_list = []
    for idx, key in enumerate(values):
        if(idx > num-1):
            array_list.append(key)
    return array_list

In [28]:
def high_corrolation_list_comb(num):
    corrolation = pd.read_csv('Comb_Correlation/continuous_f1_score_comb.csv')
    cols = corrolation.columns.tolist()
    values = {}
    for x in cols:
        values[x] = float(corrolation[x])

    values = dict(sorted(values.items(), key=lambda item: item[1], reverse=True))
    array_list = []
    for idx, key in enumerate(values):
        if(idx > num-1):
            array_list.append(key)
    return array_list

In [24]:
remove_columns_list = high_corrolation_list_peaking_separate(8,'pKmumu_piTop')
data_frame = remove_columns(total_data,remove_columns_list)
print(data_frame.columns.tolist())

['Pi_ProbNNk', 'Pi_ProbNNpi', 'Pi_ProbNNp', 'Pi_P', 'Pi_PE', 'B0_M', 'Kstar_M', 'B0_OWNPV_Y']


In [27]:
remove_columns_list = high_corrolation_list_peaking_together(13)
data_frame = remove_columns(total_data,remove_columns_list)
print(data_frame.columns.tolist())

['K_ProbNNk', 'K_ProbNNpi', 'K_ProbNNp', 'Pi_ProbNNk', 'Pi_ProbNNpi', 'Pi_ProbNNp', 'Pi_PT', 'Pi_PX', 'Pi_IPCHI2_OWNPV', 'B0_M', 'Kstar_M', 'costhetal', 'costhetak']


In [30]:
remove_columns_list = high_corrolation_list_comb(25)
remove_columns_list.append('B0_M')
data_frame = remove_columns(total_data,remove_columns_list)
print(data_frame.columns.tolist())

['mu_plus_ProbNNk', 'mu_plus_ProbNNe', 'mu_plus_ProbNNp', 'mu_minus_ProbNNk', 'mu_minus_ProbNNe', 'mu_minus_ProbNNp', 'K_ProbNNk', 'K_ProbNNpi', 'K_ProbNNmu', 'K_ProbNNe', 'Pi_ProbNNk', 'Pi_ProbNNmu', 'Pi_ProbNNe', 'Pi_ProbNNp', 'Pi_PT', 'Pi_IPCHI2_OWNPV', 'B0_ENDVERTEX_CHI2', 'B0_PT', 'Kstar_M', 'J_psi_M', 'B0_IPCHI2_OWNPV', 'B0_DIRA_OWNPV', 'B0_OWNPV_X', 'B0_OWNPV_Y', 'q2']
